In [1]:
import sys
import os
import re
import json
import glob
import copy
import time
import random
import shutil
from pathlib import Path

sys.path.append('../simulation/')
sys.path.append('../dataset_utils/')

import augmentation_utils

In [2]:
def get_program_file(file_path):
    program_name = ''
    list_of_steps = []
    i = 1
    with open(file_path, "r") as f:
        lines = f.readlines()
        program_name = lines[0].replace("\n", "")
        for line in lines[2:]:
            line = line.replace("\n", "")
            if line.startswith("<"):
                line = line.replace("<char0> ","")
                i+=1
                temp_dict = {}
                temp_dict['step']=i
                action, objs, indexs = augmentation_utils.parseStrBlock(line)
                temp_dict['action']=action
                if len(objs) > 0:
                    temp_dict['obj1']=objs[0]
                    temp_dict['index1']=indexs[0]
                if len(objs) > 1:
                    temp_dict['obj2']=objs[1]
                    temp_dict['index2']=indexs[1]
                list_of_steps.append(temp_dict)
            else:
                pass

    return program_name, list_of_steps

In [3]:
def read_and_write(script_dict,path,scene,program_name,item,start_step_index=None,put_id=None):
    if start_step_index is None:
        json_file = "/activityList-graph-state-{:03d}.json".format(script_dict['step'])
    else:
        json_file = "/activityList-graph-state-{:03d}.json".format(start_step_index)
    json_path = path+"/"+scene+"/graph states/"+program_name+json_file
    new_data={}
    with open(json_path, "r") as f:
        json_data = json.load(f)
        for node in json_data['nodes']:
            if start_step_index is None:
                if node['id']==int(script_dict['index1']) and node['class_name']==script_dict['obj1']:
                    if 'GRABED' in node['states']:
                        node['states'].remove('GRABED')
            else:
                if node['id']==int(put_id):
                    if 'GRABED' in node['states']:
                        node['states'].remove('GRABED')
        new_data=json_data
    output_path = "../simulation/Output/edited jsons/"+item+"/"+scene+"/graph states/"+program_name
    if os.path.exists(output_path) is False:
        os.makedirs(output_path)
    output_json_path = output_path+json_file
    with open(output_json_path, "w") as out_file:
        json.dump(new_data, out_file, indent=2)

In [5]:
data_path = "E:/件名：納品物/4(3)の生成データ 一式/データセット２（部屋から移動あり）/"
for item in os.listdir(data_path):
    category_path = data_path+item
    for scene in os.listdir(category_path+"/executed_program/"):
        program_path = category_path+"/executed_program/"+scene+"/"
        for program in os.listdir(program_path):
            path=program_path+program
            program_name, script_info = get_program_file(path)
            keep_change_list=[]
            for script_dict in script_info:
                if script_dict['action'] in ['GRAB']:
                    for dict in keep_change_list:
                        if script_dict['index1'] == dict['put_id']:
                            keep_change_list.remove(dict)
                # we need to change JSON file until program end or agent pick same obj again
                if len(keep_change_list) > 0:
                    existing_path=category_path
                    for dict in keep_change_list:
                        dict['start_index'] = dict['start_index']+1
                        read_and_write(script_dict,existing_path,scene,program_name,item, dict['start_index'],dict['put_id'])
                        existing_path="../simulation/Output/edited jsons/"+item
                        json_file = "/activityList-graph-state-{:03d}.json".format(dict['start_index'])
                        if os.path.exists(existing_path+"/"+scene+"/graph states/"+program_name+json_file) is False:
                            print('edit new',program_name+json_file)
                            existing_path=category_path
                if script_dict['action'] in ['PUTBACK','PUTOBJBACK','PUTIN']:
                    keep_change_dict={}
                    keep_change_dict['start_index']=script_dict['step']
                    keep_change_dict['put_id']=script_dict['index1']
                    keep_change_list.append(keep_change_dict)
                    if len(keep_change_list) > 1:
                        existing_path="../simulation/Output/edited jsons/"+item
                        json_file = "/activityList-graph-state-{:03d}.json".format(script_dict['step'])
                        if os.path.exists(existing_path+"/"+scene+"/graph states/"+program_name+json_file):
                            read_and_write(script_dict,existing_path,scene,program_name,item)
                        else:
                            read_and_write(script_dict,category_path,scene,program_name,item)
                    else:
                        read_and_write(script_dict,category_path,scene,program_name,item)